In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta,date
import re

In [4]:
path = r"C:\Users\NT550-052\OneDrive\바탕 화면\ASAC\프로젝트\ML\web_crawling\CGW\data_preprocessing\mid_data\csvs\steam_store_df_2018.csv"

data = pd.read_csv(path,sep=";")

data.drop(columns="Unnamed: 0",inplace=True)

data

,appid,required_age,is_free,controller_support,dlc,metacritic,windows,mac,linux,platforms_num,recommendations,achievements,release_date,release_date_final
0,294100,0.0,0.0,NaN,6.0,87.0,1.0,1.0,1.0,3.0,134967.0,NaN,2018-Oct-17,2018-10-17
1,960090,0.0,0.0,NaN,1.0,NaN,1.0,1.0,0.0,2.0,222889.0,146.0,2018-Dec-17,2018-12-17
2,504230,0.0,0.0,full,1.0,88.0,1.0,1.0,1.0,3.0,66397.0,32.0,2018-Jan-25,2018-01-25
3,588650,0.0,0.0,full,7.0,89.0,1.0,1.0,1.0,3.0,113184.0,121.0,2018-ug-6A,NaN
4,264710,0.0,0.0,full,1.0,87.0,1.0,1.0,0.0,2.0,208334.0,17.0,2018-Jan-23,2018-01-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7708,836860,0.0,False,NaN,NaN,NaN,1.0,0.0,0.0,1.0,NaN,NaN,2018-Jul-13,2018-07-13
7709,792990,0.0,False,NaN,NaN,NaN,1.0,0.0,0.0,1.0,561.0,NaN,2018-Nov-30,2018-11-30
7710,785680,0.0,False,NaN,NaN,NaN,1.0,1.0,0.0,2.0,NaN,26.0,2018-Feb-27,2018-02-27
7711,862520,0.0,False,NaN,NaN,NaN,1.0,0.0,0.0,1.0,NaN,NaN,2018-ug-7A,NaN


In [5]:
# 메타크리틱,컨트롤러 0/1컬럼 만들기
data["metacritic_tf"] = data["metacritic"].notnull().astype("int")
data["controller_support"] = data["controller_support"].notnull().astype("int")
data = data[["appid","required_age","is_free","controller_support","dlc","metacritic","metacritic_tf","windows","mac","linux","platforms_num","recommendations","achievements","release_date","release_date_final"]]

In [7]:
# release_date 합치기
date_path = r"C:\Users\NT550-052\OneDrive\바탕 화면\ASAC\프로젝트\ML\web_crawling\CGW\data_handling\csvs\2018_date.csv"
date = pd.read_csv(date_path)
data.drop(columns=["release_date","release_date_final"],inplace=True)
data = pd.merge(data,date,how="left",on="appid")
data['new_date'] = pd.to_datetime(data["new_date"], format='%Y-%m-%d')
today = "2023-05-19"
today = datetime.strptime(today,'%Y-%m-%d')
data["days_after_release"] = data["new_date"].apply(lambda x: (today - x).days)

c:\Users\NT550-052\anaconda3\envs\mlp\lib\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
data["required_age"].fillna(0,inplace=True)
data["required_age"] = data["required_age"].astype("str")
type(data["required_age"][0])

str

In [9]:
for idx,elem in enumerate(data["required_age"]):
    if "+" in elem:
        data["required_age"][idx] = elem.replace("+","")
    elif elem == "ALL":
        data["required_age"][idx] = "0"

In [10]:
# 장르 범주 나누기
# nan -> 0, 17,18,19 -> 18, 14,15,16 -> 15, 나머지 12

data["required_age"] = data["required_age"].astype("float")
for idx,elem in enumerate(data["required_age"]):
    if elem > 16:
        data["required_age"][idx] = 18
    elif elem > 13:
        data["required_age"][idx] = 15
    elif elem > 0:
        data["required_age"][idx] = 12
data["required_age"].value_counts()

c:\Users\NT550-052\anaconda3\envs\mlp\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
c:\Users\NT550-052\anaconda3\envs\mlp\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
c:\Users\NT550-052\anaconda3\envs\mlp\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

0.0     7633
18.0      58
12.0      15
15.0       7
Name: required_age, dtype: int64

In [11]:
# 추천수 nan, 도전과제 nan, dlc nan을 0으로 
data["recommendations"].fillna(0,inplace=True)
data["achievements"].fillna(0,inplace=True)
data["dlc"].fillna(0,inplace=True)
print(sum(data["recommendations"].isnull()))
print(sum(data["achievements"].isnull()))
print(sum(data["dlc"].isnull()))

0
0
0


In [12]:
data.rename(columns={"new_date":"release_date"},inplace=True)

In [13]:
data = data[["appid","required_age","is_free","controller_support","dlc","metacritic","metacritic_tf","windows","mac","linux","platforms_num","recommendations","achievements","release_date","days_after_release"]]

In [14]:
data

,appid,required_age,is_free,controller_support,dlc,metacritic,metacritic_tf,windows,mac,linux,platforms_num,recommendations,achievements,release_date,days_after_release
0,294100,0.0,0.0,0,6.0,87.0,1,1.0,1.0,1.0,3.0,134967.0,0.0,2018-10-17,1675.0
1,960090,0.0,0.0,0,1.0,NaN,0,1.0,1.0,0.0,2.0,222889.0,146.0,2018-12-18,1613.0
2,504230,0.0,0.0,1,1.0,88.0,1,1.0,1.0,1.0,3.0,66397.0,32.0,2018-01-25,1940.0
3,588650,0.0,0.0,1,7.0,89.0,1,1.0,1.0,1.0,3.0,113184.0,121.0,2018-08-06,1747.0
4,264710,0.0,0.0,1,1.0,87.0,1,1.0,1.0,0.0,2.0,208334.0,17.0,2018-01-24,1941.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7708,836860,0.0,False,0,0.0,NaN,0,1.0,0.0,0.0,1.0,0.0,0.0,2018-07-13,1771.0
7709,792990,0.0,False,0,0.0,NaN,0,1.0,0.0,0.0,1.0,561.0,0.0,2018-12-01,1630.0
7710,785680,0.0,False,0,0.0,NaN,0,1.0,1.0,0.0,2.0,0.0,26.0,2018-02-27,1907.0
7711,862520,0.0,False,0,0.0,NaN,0,1.0,0.0,0.0,1.0,0.0,0.0,2018-08-08,1745.0


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7713 entries, 0 to 7712
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   appid               7713 non-null   int64         
 1   required_age        7713 non-null   float64       
 2   is_free             7670 non-null   object        
 3   controller_support  7713 non-null   int32         
 4   dlc                 7713 non-null   float64       
 5   metacritic          332 non-null    float64       
 6   metacritic_tf       7713 non-null   int32         
 7   windows             7670 non-null   float64       
 8   mac                 7670 non-null   float64       
 9   linux               7670 non-null   float64       
 10  platforms_num       7670 non-null   float64       
 11  recommendations     7713 non-null   float64       
 12  achievements        7713 non-null   float64       
 13  release_date        7635 non-null   datetime64[n

In [ ]:
# save_path = f"C:\Users\NT550-052\OneDrive\바탕 화면\ASAC\프로젝트\ML\web_crawling\CGW\data_preprocessing\mid_data\final\steam_store_after_pcs_2018.csv"
# data.to_csv(save_path,sep=";")